## Step 1: Remove unused conflicting packages, install Gen-Ai/ChromaDB package

In [1]:
!pip uninstall -qqy jupyterlab kfp 
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

In [2]:
from google import genai
from google.genai import types
from kaggle_secrets import UserSecretsClient
import enum
from IPython.display import Markdown, display
import chromadb



genai.__version__

'1.7.0'

In [3]:

client = genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY"))

for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [4]:
DOCUMENT1 = "AI is revolutionizing medical imaging. Machine learning algorithms can analyze X-rays, MRIs, and CT scans with greater speed and accuracy than human radiologists, detecting subtle anomalies that might otherwise be missed. This leads to earlier diagnoses of diseases like cancer and Alzheimer's, improving patient outcomes and saving lives."

DOCUMENT2 = "Personalized medicine is becoming a reality thanks to AI. By analyzing a patient's genetic data, medical history, and lifestyle, AI systems can predict their risk of developing certain diseases and tailor treatment plans to their individual needs. This approach promises to optimize drug efficacy and minimize side effects, leading to more effective and targeted therapies."

DOCUMENT3 = "AI-powered chatbots and virtual assistants are transforming patient care by providing 24/7 access to medical information and support. These tools can answer common health questions, schedule appointments, and even monitor chronic conditions remotely. This reduces the burden on healthcare providers and empowers patients to take a more active role in managing their own health."


documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## Setup embedding function (Gemini's way to turn text into numbers)


In [5]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

### Sets up a Chroma database using Gemini embeddings to store and search text based on its meaning.

In [6]:

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True #used for long texts(docs/articles/pdf)

# Create a Chroma database client
chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# Add some documents (your text data)
db.add(documents=documents, ids=[str(i) for i in range(len(documents))]) 


In [7]:
db.count()
# You can peek at the data too.
# db.peek(1)

3

### Retrieval: Find relevant documents¶
To search the Chroma database, call the query method. Note that you also switch to the retrieval_query mode of embedding generation.

In [10]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False # default (used for sentences/word)

queries = [
    "How does AI help with personalized medicine?",
    "What are the benefits of using AI in medical imaging?",
    "Can AI chatbots improve patient care?"
]



In [11]:
for query in queries:
    result = db.query(query_texts=[query], n_results=1)
    [all_passages] = result["documents"]

    query_oneline = query.replace("\n", " ")

    # This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
    prompt = f"""
    You are an AI assistant designed to answer questions from provided text passages.
    Your goal is to provide clear, comprehensive answers in a friendly and conversational style, suitable for a non-technical audience.
    Please break down complex concepts into simple terms and use complete sentences. If a provided passage is irrelevant to the question, ignore it.

    QUESTION: {query_oneline}

    """

    # Add the retrieved documents to the prompt.
    for passage in all_passages:
        passage_oneline = passage.replace("\n", " ")
        prompt += f"PASSAGE: {passage_oneline}\n"

    print(prompt)
    print("---Next Query---") # add a seperator.


    You are an AI assistant designed to answer questions from provided text passages.
    Your goal is to provide clear, comprehensive answers in a friendly and conversational style, suitable for a non-technical audience.
    Please break down complex concepts into simple terms and use complete sentences. If a provided passage is irrelevant to the question, ignore it.

    QUESTION: How does AI help with personalized medicine?

    PASSAGE: Personalized medicine is becoming a reality thanks to AI. By analyzing a patient's genetic data, medical history, and lifestyle, AI systems can predict their risk of developing certain diseases and tailor treatment plans to their individual needs. This approach promises to optimize drug efficacy and minimize side effects, leading to more effective and targeted therapies.

---Next Query---

    You are an AI assistant designed to answer questions from provided text passages.
    Your goal is to provide clear, comprehensive answers in a friendly and 